In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder


from tqdm.notebook import tqdm

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, roc_curve
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, BaggingClassifier

# ustawia domyślną wielkość wykresów
plt.rcParams['figure.figsize'] = (12,8)
# to samo tylko dla tekstu
plt.rcParams['font.size'] = 16


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# wczytanie danych
train = pd.read_csv('E:/data/airline_passenger_satisfaction/train.csv')
test = pd.read_csv('E:/data/airline_passenger_satisfaction/test.csv')

## Obróbka danych

In [3]:
train.columns

Index(['Unnamed: 0', 'id', 'Gender', 'Customer Type', 'Age', 'Type of Travel',
       'Class', 'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

In [4]:
train.isnull().sum()

Unnamed: 0                             0
id                                     0
Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             310
satisfaction    

In [5]:
test.isnull().sum()

Unnamed: 0                            0
id                                    0
Gender                                0
Customer Type                         0
Age                                   0
Type of Travel                        0
Class                                 0
Flight Distance                       0
Inflight wifi service                 0
Departure/Arrival time convenient     0
Ease of Online booking                0
Gate location                         0
Food and drink                        0
Online boarding                       0
Seat comfort                          0
Inflight entertainment                0
On-board service                      0
Leg room service                      0
Baggage handling                      0
Checkin service                       0
Inflight service                      0
Cleanliness                           0
Departure Delay in Minutes            0
Arrival Delay in Minutes             83
satisfaction                          0


In [6]:
train = train.dropna()
test = test.dropna()

In [7]:
ord_enc = OrdinalEncoder()

train = train.drop(columns = ['Unnamed: 0', 'id', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness'])

train["Gender"] = ord_enc.fit_transform(train[["Gender"]])
train["Customer Type"] = ord_enc.fit_transform(train[["Customer Type"]])
train["Type of Travel"] = ord_enc.fit_transform(train[["Type of Travel"]])
train["satisfaction"] = ord_enc.fit_transform(train[["satisfaction"]])
train["Class"] = ord_enc.fit_transform(train[["Class"]])




test = test.drop(columns = ['Unnamed: 0', 'id', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness'])

test["Gender"] = ord_enc.fit_transform(test[["Gender"]])
test["Customer Type"] = ord_enc.fit_transform(test[["Customer Type"]])
test["Type of Travel"] = ord_enc.fit_transform(test[["Type of Travel"]])
test["satisfaction"] = ord_enc.fit_transform(test[["satisfaction"]])
test["Class"] = ord_enc.fit_transform(test[["Class"]])

In [8]:
train

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1.0,0.0,13,1.0,2.0,460,25,18.0,0.0
1,1.0,1.0,25,0.0,0.0,235,1,6.0,0.0
2,0.0,0.0,26,0.0,0.0,1142,0,0.0,1.0
3,0.0,0.0,25,0.0,0.0,562,11,9.0,0.0
4,1.0,0.0,61,0.0,0.0,214,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
103899,0.0,1.0,23,0.0,1.0,192,3,0.0,0.0
103900,1.0,0.0,49,0.0,0.0,2347,0,0.0,1.0
103901,1.0,1.0,30,0.0,0.0,1995,7,14.0,0.0
103902,0.0,1.0,22,0.0,1.0,1000,0,0.0,0.0


## Podział danych

In [9]:
X_train = train.drop(columns=['satisfaction'])
X_test = test.drop(columns=['satisfaction'])

y_train = train.satisfaction
y_test = test.satisfaction

In [10]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, random_state=420, train_size=0.66)

In [11]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

print(X_val.shape)
print(y_val.shape)

(103594, 8)
(103594,)
(17089, 8)
(17089,)
(8804, 8)
(8804,)


## Random Forest

In [12]:
rf = RandomForestClassifier()

In [13]:
rf.fit(X_train, y_train).score(X_test, y_test)

0.7619521329510212

In [14]:
#przetestujmy jak działa to dla różnych parametrów
scores = []
best = [0,0,0,0]
for i in [1, 2, 4, 8, 16, 32, 64, 128]:
    for j in  [1, 2, 4, 8, 16, 32, 64, 128]:
        for k in  [2, 4, 8, 16, 32, 64, 128]:
            rf = RandomForestClassifier(max_depth = i, n_estimators = j, max_leaf_nodes = k)
            score = rf.fit(X_train, y_train).score(X_test, y_test)
            print (f"max_depth: {i},  n_estimators: {j}, max_leaf_nodes: {k}, score: {score}")
            if (score > best[3]):
                best = [i, j, k, score]
            scores.append(score)
            

max_depth: 1,  n_estimators: 1, max_leaf_nodes: 2, score: 0.5603604657967113
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 4, score: 0.6616536953595881
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 8, score: 0.7476739423020656
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 16, score: 0.6616536953595881
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 32, score: 0.683129498507812
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 64, score: 0.5603604657967113
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 128, score: 0.7476739423020656
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 2, score: 0.76259582187372
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 4, score: 0.7742992568318802
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 8, score: 0.7476739423020656
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 16, score: 0.76259582187372
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 32, score: 0.7476739423020656
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 64, score: 0.7636491310199

max_depth: 2,  n_estimators: 128, max_leaf_nodes: 2, score: 0.7700860202469425
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 4, score: 0.7710222950435953
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 8, score: 0.7697349171981976
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 16, score: 0.7710222950435953
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 32, score: 0.7708467435192229
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 64, score: 0.7703786061208965
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 128, score: 0.7706126748200597
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 2, score: 0.683129498507812
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 4, score: 0.6953595880390895
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 8, score: 0.7785124934168178
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 16, score: 0.6870501492187957
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 32, score: 0.7585581368131546
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 64, scor

max_depth: 8,  n_estimators: 64, max_leaf_nodes: 2, score: 0.7618350986014395
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 4, score: 0.7751770144537422
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 8, score: 0.778746562115981
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 16, score: 0.786119726139622
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 32, score: 0.7906255485985136
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 64, score: 0.7979987126221546
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 128, score: 0.7971209550002926
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 2, score: 0.775235531628533
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 4, score: 0.775235531628533
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 8, score: 0.7783369418924454
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 16, score: 0.7882263444320908
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 32, score: 0.7889870677043712
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 64, sc

max_depth: 32,  n_estimators: 16, max_leaf_nodes: 128, score: 0.794955819533033
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 2, score: 0.770203054596524
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 4, score: 0.7717245011410849
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 8, score: 0.779916905611797
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 16, score: 0.7867634150623208
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 32, score: 0.7929662355901457
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 64, score: 0.7974135408742465
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 128, score: 0.7975305752238282
max_depth: 32,  n_estimators: 64, max_leaf_nodes: 2, score: 0.7680964363040552
max_depth: 32,  n_estimators: 64, max_leaf_nodes: 4, score: 0.7750599801041606
max_depth: 32,  n_estimators: 64, max_leaf_nodes: 8, score: 0.7785124934168178
max_depth: 32,  n_estimators: 64, max_leaf_nodes: 16, score: 0.7869389665866932
max_depth: 32,  n_estimators: 64, max_leaf_node

max_depth: 128,  n_estimators: 8, max_leaf_nodes: 64, score: 0.7906255485985136
max_depth: 128,  n_estimators: 8, max_leaf_nodes: 128, score: 0.7983498156708994
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 2, score: 0.7608403066299959
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 4, score: 0.7774591842705835
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 8, score: 0.7783954590672363
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 16, score: 0.7869389665866932
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 32, score: 0.7840716250219439
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 64, score: 0.7930832699397273
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 128, score: 0.7998712622154602
max_depth: 128,  n_estimators: 32, max_leaf_nodes: 2, score: 0.753467142606355
max_depth: 128,  n_estimators: 32, max_leaf_nodes: 4, score: 0.7686230908771724
max_depth: 128,  n_estimators: 32, max_leaf_nodes: 8, score: 0.777576218620165
max_depth: 128,  n_estimators: 32, m

In [15]:
print (f"mediana: {np.median(scores)}")
print (f"srednia: {np.mean(scores)}")
print (f"najlepszy: max_depth: {best[0]}, n_estimators: {best[1]}, max_leaf_nodes: {best[2]}, score: {best[3]}")

mediana: 0.774796652817602
srednia: 0.7676302581861348
najlepszy: max_depth: 128, n_estimators: 128, max_leaf_nodes: 128, score: 0.8000468137398327


## XGBoost

In [16]:
xgb_cls = xgb.XGBClassifier()

In [17]:
xgb_cls.fit(X_train, y_train, verbose=True, eval_metric="error", eval_set=[(X_val, y_val)])

[0]	validation_0-error:0.19945
[1]	validation_0-error:0.19764
[2]	validation_0-error:0.19798
[3]	validation_0-error:0.19752
[4]	validation_0-error:0.19650
[5]	validation_0-error:0.19616
[6]	validation_0-error:0.19662
[7]	validation_0-error:0.19616
[8]	validation_0-error:0.19548
[9]	validation_0-error:0.19662
[10]	validation_0-error:0.19559


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11]	validation_0-error:0.19605
[12]	validation_0-error:0.19593
[13]	validation_0-error:0.19503
[14]	validation_0-error:0.19480
[15]	validation_0-error:0.19525
[16]	validation_0-error:0.19559
[17]	validation_0-error:0.19571
[18]	validation_0-error:0.19559
[19]	validation_0-error:0.19571
[20]	validation_0-error:0.19593
[21]	validation_0-error:0.19582
[22]	validation_0-error:0.19559
[23]	validation_0-error:0.19548
[24]	validation_0-error:0.19559
[25]	validation_0-error:0.19571
[26]	validation_0-error:0.19559
[27]	validation_0-error:0.19559
[28]	validation_0-error:0.19559
[29]	validation_0-error:0.19559
[30]	validation_0-error:0.19571
[31]	validation_0-error:0.19571
[32]	validation_0-error:0.19605
[33]	validation_0-error:0.19616
[34]	validation_0-error:0.19639
[35]	validation_0-error:0.19662
[36]	validation_0-error:0.19684
[37]	validation_0-error:0.19673
[38]	validation_0-error:0.19650
[39]	validation_0-error:0.19605
[40]	validation_0-error:0.19593
[41]	validation_0-error:0.19605
[42]	val

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=24,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [18]:
xgb_cls.score(X_test, y_test)

0.8006905026625315

In [19]:
#przetestujmy jak działa to dla różnych parametrów
scores = []
best = [0,0,0,0,0]
for i in [1, 4, 16, 64, 128]:
    for j in  [1, 4, 16, 64, 128]:
        for k in  [4, 16, 64, 128]:
            for l in [1, 4, 16, 64, 128]:
                
                xgb_cls = xgb.XGBClassifier(objective="binary:logistic", 
                                            seed = 42, 
                                            use_label_encoder=False, 
                                            max_depth = i, 
                                            n_estimators = j, 
                                            max_leaf_nodes = k, 
                                            max_leaves = l,
                                            verbocity = 0,
                                            silent = True,
                                            verbosity = 0,
                                            verbose=0)

                score = xgb_cls.fit(X_train, y_train, verbose = False).score(X_test, y_test)
                print (f"max_depth: {i},  n_estimators: {j}, max_leaf_nodes: {k}, max_leaves: {l}, score: {score}")
                if (score > best[4]):
                    best = [i, j, k, l, score]
                scores.append(score)

max_depth: 1,  n_estimators: 1, max_leaf_nodes: 4, max_leaves: 1, score: 0.7476739423020656
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 4, max_leaves: 4, score: 0.7476739423020656
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 4, max_leaves: 16, score: 0.7476739423020656
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 4, max_leaves: 64, score: 0.7476739423020656
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 4, max_leaves: 128, score: 0.7476739423020656
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 16, max_leaves: 1, score: 0.7476739423020656
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 16, max_leaves: 4, score: 0.7476739423020656
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 16, max_leaves: 16, score: 0.7476739423020656
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 16, max_leaves: 64, score: 0.7476739423020656
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 16, max_leaves: 128, score: 0.7476739423020656
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 64, max_leaves: 1, 

max_depth: 1,  n_estimators: 128, max_leaf_nodes: 16, max_leaves: 16, score: 0.78963075662707
max_depth: 1,  n_estimators: 128, max_leaf_nodes: 16, max_leaves: 64, score: 0.78963075662707
max_depth: 1,  n_estimators: 128, max_leaf_nodes: 16, max_leaves: 128, score: 0.78963075662707
max_depth: 1,  n_estimators: 128, max_leaf_nodes: 64, max_leaves: 1, score: 0.78963075662707
max_depth: 1,  n_estimators: 128, max_leaf_nodes: 64, max_leaves: 4, score: 0.78963075662707
max_depth: 1,  n_estimators: 128, max_leaf_nodes: 64, max_leaves: 16, score: 0.78963075662707
max_depth: 1,  n_estimators: 128, max_leaf_nodes: 64, max_leaves: 64, score: 0.78963075662707
max_depth: 1,  n_estimators: 128, max_leaf_nodes: 64, max_leaves: 128, score: 0.78963075662707
max_depth: 1,  n_estimators: 128, max_leaf_nodes: 128, max_leaves: 1, score: 0.78963075662707
max_depth: 1,  n_estimators: 128, max_leaf_nodes: 128, max_leaves: 4, score: 0.78963075662707
max_depth: 1,  n_estimators: 128, max_leaf_nodes: 128, max_l

max_depth: 4,  n_estimators: 64, max_leaf_nodes: 64, max_leaves: 128, score: 0.8023875007314647
max_depth: 4,  n_estimators: 64, max_leaf_nodes: 128, max_leaves: 1, score: 0.8023875007314647
max_depth: 4,  n_estimators: 64, max_leaf_nodes: 128, max_leaves: 4, score: 0.8023875007314647
max_depth: 4,  n_estimators: 64, max_leaf_nodes: 128, max_leaves: 16, score: 0.8023875007314647
max_depth: 4,  n_estimators: 64, max_leaf_nodes: 128, max_leaves: 64, score: 0.8023875007314647
max_depth: 4,  n_estimators: 64, max_leaf_nodes: 128, max_leaves: 128, score: 0.8023875007314647
max_depth: 4,  n_estimators: 128, max_leaf_nodes: 4, max_leaves: 1, score: 0.8019778805079291
max_depth: 4,  n_estimators: 128, max_leaf_nodes: 4, max_leaves: 4, score: 0.8019778805079291
max_depth: 4,  n_estimators: 128, max_leaf_nodes: 4, max_leaves: 16, score: 0.8019778805079291
max_depth: 4,  n_estimators: 128, max_leaf_nodes: 4, max_leaves: 64, score: 0.8019778805079291
max_depth: 4,  n_estimators: 128, max_leaf_node

max_depth: 16,  n_estimators: 64, max_leaf_nodes: 4, max_leaves: 1, score: 0.7886944818304172
max_depth: 16,  n_estimators: 64, max_leaf_nodes: 4, max_leaves: 4, score: 0.7886944818304172
max_depth: 16,  n_estimators: 64, max_leaf_nodes: 4, max_leaves: 16, score: 0.7886944818304172
max_depth: 16,  n_estimators: 64, max_leaf_nodes: 4, max_leaves: 64, score: 0.7886944818304172
max_depth: 16,  n_estimators: 64, max_leaf_nodes: 4, max_leaves: 128, score: 0.7886944818304172
max_depth: 16,  n_estimators: 64, max_leaf_nodes: 16, max_leaves: 1, score: 0.7886944818304172
max_depth: 16,  n_estimators: 64, max_leaf_nodes: 16, max_leaves: 4, score: 0.7886944818304172
max_depth: 16,  n_estimators: 64, max_leaf_nodes: 16, max_leaves: 16, score: 0.7886944818304172
max_depth: 16,  n_estimators: 64, max_leaf_nodes: 16, max_leaves: 64, score: 0.7886944818304172
max_depth: 16,  n_estimators: 64, max_leaf_nodes: 16, max_leaves: 128, score: 0.7886944818304172
max_depth: 16,  n_estimators: 64, max_leaf_node

max_depth: 64,  n_estimators: 16, max_leaf_nodes: 16, max_leaves: 4, score: 0.7747673942302066
max_depth: 64,  n_estimators: 16, max_leaf_nodes: 16, max_leaves: 16, score: 0.7747673942302066
max_depth: 64,  n_estimators: 16, max_leaf_nodes: 16, max_leaves: 64, score: 0.7747673942302066
max_depth: 64,  n_estimators: 16, max_leaf_nodes: 16, max_leaves: 128, score: 0.7747673942302066
max_depth: 64,  n_estimators: 16, max_leaf_nodes: 64, max_leaves: 1, score: 0.7747673942302066
max_depth: 64,  n_estimators: 16, max_leaf_nodes: 64, max_leaves: 4, score: 0.7747673942302066
max_depth: 64,  n_estimators: 16, max_leaf_nodes: 64, max_leaves: 16, score: 0.7747673942302066
max_depth: 64,  n_estimators: 16, max_leaf_nodes: 64, max_leaves: 64, score: 0.7747673942302066
max_depth: 64,  n_estimators: 16, max_leaf_nodes: 64, max_leaves: 128, score: 0.7747673942302066
max_depth: 64,  n_estimators: 16, max_leaf_nodes: 128, max_leaves: 1, score: 0.7747673942302066
max_depth: 64,  n_estimators: 16, max_lea

max_depth: 128,  n_estimators: 4, max_leaf_nodes: 64, max_leaves: 16, score: 0.7854760372169232
max_depth: 128,  n_estimators: 4, max_leaf_nodes: 64, max_leaves: 64, score: 0.7854760372169232
max_depth: 128,  n_estimators: 4, max_leaf_nodes: 64, max_leaves: 128, score: 0.7854760372169232
max_depth: 128,  n_estimators: 4, max_leaf_nodes: 128, max_leaves: 1, score: 0.7854760372169232
max_depth: 128,  n_estimators: 4, max_leaf_nodes: 128, max_leaves: 4, score: 0.7854760372169232
max_depth: 128,  n_estimators: 4, max_leaf_nodes: 128, max_leaves: 16, score: 0.7854760372169232
max_depth: 128,  n_estimators: 4, max_leaf_nodes: 128, max_leaves: 64, score: 0.7854760372169232
max_depth: 128,  n_estimators: 4, max_leaf_nodes: 128, max_leaves: 128, score: 0.7854760372169232
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 4, max_leaves: 1, score: 0.7748844285797881
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 4, max_leaves: 4, score: 0.7748844285797881
max_depth: 128,  n_estimators: 16, ma

In [20]:
print (f"mediana: {np.median(scores)}")
print (f"srednia: {np.mean(scores)}")
print (f"najlepszy: max_depth: {best[0]}, n_estimators: {best[1]}, max_leaf_nodes: {best[2]}, max_leaves: {best[3]}, score: {best[4]}")

mediana: 0.7841301421967347
srednia: 0.7811691731523202
najlepszy: max_depth: 4, n_estimators: 64, max_leaf_nodes: 4, max_leaves: 1, score: 0.8023875007314647


## Wnioski 
* Z XGBoost jest lepszy z parametrami domyślnymi niż Random Forest (także z domyślnymi).

* XGBoost okazuje się być średnio lepszy niż Random Forest.

* W przypadku obu modeli dobranie większych parametrów pozytywnie wpływa na score.

* Im więcej estymatorów oraz im większa głębokość, tym wolniej się trenuje (oba modele).

* Najlepsze u mnie okazały się parametry:
    * RF: max_depth: 128, n_estimators: 128, max_leaf_nodes: 128 || (score: 0.8000468137398327)
    * XGB: max_depth: 4, n_estimators: 64, max_leaf_nodes: 4, max_leaves: 1 || (score: 0.8023875007314647)